# Python en la vida cotidiana

## Mensajes de WhatsApp

Para este ejercicio se usó de referencia un tutorial sobre el [envío de WhatsApps desde python](https://www.analyticslane.com/2022/03/07/enviar-mensajes-de-whatsapp-con-python/).

### Instalación de paquetes

In [9]:
import pywhatkit as kit # Para enviar mensajes por WhatsApp
import sys # Para establecer las rutas de acceso

# La ruta es dentro del sistema del creador, cambiar a donde se tenga el módulo MensajesWhatsapp
sys.path.append(r'C:\Users\mapis\OneDrive\Documents\Universidad\5 - Quinto semestre\Herramientas para las ciencias de datos II\Proyecto individual\cod')

In [10]:
# Módulo creado
from MensajesWhatsapp import MensajesWhatsapp

Se manda un mensaje de prueba.

In [13]:
# Por la privacidad de la persona, no se deja un número escrito, en caso de ser de Costa Rica basta poner el número receptor pegado al codigo del país
numero = MensajesWhatsapp('+506')
numero.mensaje_instantaneo('Hola, acá va el mensaje')

## Automatización de procesos en excel

Para la realización de este ejercicio se buscó la documentación de la librería [xlsxwriter](https://xlsxwriter.readthedocs.io), la cual trae múltiples ejemplos. La idea del ejercicio es crear una clase que permita automatizar algunos de los procesos que se podrían ocupar en Excel, usando las herramientas que proporciona python.

### Instalación de paquetes

In [14]:
import xlsxwriter
import pandas as pd
import sys # Para establecer las rutas de acceso

# La ruta es dentro del sistema del creador, cambiar a donde se tenga el módulo MensajesWhatsapp
sys.path.append(r'C:\Users\mapis\OneDrive\Documents\Universidad\5 - Quinto semestre\Herramientas para las ciencias de datos II\Proyecto individual\cod')

In [15]:
from EditorExcel import EditorExcel

### Prueba

In [17]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Necesario para cerrar el escritor, si no se pone, no es posible abrir el archivo
writer.close()

# Web scraping

Para este proceso se usó como referencia un tutorial para hacer [web scraping](https://nanonets.com/blog/web-scraping-with-python-tutorial/).

### Instalación de paquetes

In [1]:
import requests # Para hacer solicitudes al sitio web
from bs4 import BeautifulSoup # Para analizar el código del HTML
import pandas as pd
import time # Para añadir un cierto delay para no sobrecargar la página con las solucitudes
import numpy as np

### Extracción de los datos

Se inicia el proceso de extracción de datos, para esto se eligió una página web de [compra/venta de carros](https://crautos.com/autosusados/), se usará el comando [find_all](https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/#scraping-bs-pasos) de la librería BeautifulSoup, además, se buscó la manera de hacer casting de [string a float](https://ellibrodepython.com/casting-python#:~:text=class%20%27str%27>-,Convertir%20string%20a%20float,a%20float%20usando%20float()%20.) pues se ocupará para los precios.

In [2]:
# Se selecciona el url del que se extraerá la información
url = "https://crautos.com/autosusados/"

# Manda una solicitud para obtener la información de la página web (librería requests)
respuesta = requests.get(url)

# Se analiza la información del HTML (librería BeautifulSoup)
sopa = BeautifulSoup(respuesta.content, 'html.parser')

#### Funciones necesarias

Se procede a crear las funciones para extraer cada uno de los elementos necesarios de la página web.

In [3]:
def elementos_simples(sopa_bs4, tipo, clase):
    '''
    Función que extrae los elementos que no ocupan modificaciones de un HTML

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_simple : list
        Lista con los elementos encontrados dentro del HTML correspondiente
    '''
    # Lista para guardar los elementos correspondientes
    lista_simple = []

    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in sopa_bs4.find_all(tipo, class_ = clase):
        # Se extrae el elemento y se agrega a la lista
        elemento = fila.get_text()
        lista_simple.append(elemento)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_simple

In [4]:
def elemento_anno(sopa_bs4, tipo, clase):
    '''
    Función que extrae los años de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_annos : list
        Lista con los años encontrados dentro del HTML correspondiente
    '''
    # Lista para guardar los elementos correspondientes
    lista_annos = []

    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in sopa_bs4.find_all(tipo, class_ = clase):
        # Extrae una cadena con muchos espacios, solo nos interesa el año (los primeros 4 caracteres)
        anno = fila.find('b').get_text()[ : 4]
        lista_annos.append(anno)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_annos

In [5]:
def elemento_modelos(sopa_bs4, tipo, clase):
    '''
    Función que extrae los modelos de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_modelos : list
        Lista con los modelos de carro encontrados dentro de la página web de crautos
    '''
    # Lista para guardar los elementos correspondientes
    lista_modelos = []

    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in sopa_bs4.find_all(tipo, class_ = clase):
        # Extrae toda la información detallada del carro, quita los espacios del inicio y el final, separa por guiones y guarda el primer elemento
        modelo = fila.get_text().strip().split(' - ')[0]
        lista_modelos.append(modelo)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_modelos

In [6]:
def elemento_transmision(sopa_bs4, tipo, clase):
    '''
    Función que extrae las transmisiones de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_transmisiones : list
        Lista con las transmisiones de carro encontrados dentro de la página web de crautos
    '''
    # Lista para guardar los elementos correspondientes
    lista_transmision = []

    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in sopa_bs4.find_all(tipo, class_ = clase):
        # Guardamos la fila, pues se le harán
        fila_actual = fila.get_text()
        
        # Obtenemos la transmisión
        if(fila_actual.count("Automático") > 0):
            transmision = "Automático"
        elif(fila_actual.count("Manual\r\n") > 0):
            transmision = "Manual"
        else:
            transmision = np.nan
        lista_transmision.append(transmision)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_transmision

In [7]:
def elemento_transmision(sopa_bs4, tipo, clase):
    '''
    Función que extrae las transmisiones de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_transmisiones : list
        Lista con las transmisiones de carro encontrados dentro de la página web de crautos
    '''
    # Lista para guardar los elementos correspondientes
    lista_transmision = []

    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in sopa_bs4.find_all(tipo, class_ = clase):
        # Guardamos la fila, pues se le harán
        fila_actual = fila.get_text()
        
        # Obtenemos la transmisión
        if(fila_actual.count("Automático") > 0):
            transmision = "Automático"
        elif(fila_actual.count("Manual\r\n") > 0):
            transmision = "Manual"
        else:
            transmision = np.nan
        lista_transmision.append(transmision)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_transmision

In [8]:
def elemento_precio(sopa_bs4, tipo, clase):
    '''
    Función que extrae el precio, en colones, de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_precios : list
        Lista con los precios, en colones, de cada carro encontrados dentro de la página web de crautos
    '''
    # Se inicia extrayendo el tipo de cambio
    tipo_cambio = 0
    
    for fila in sopa_bs4.find_all('div', class_ = 'brandtitle2'):
        if(fila.get_text().count("Tipo de cambio: ") > 0):
            tipo_cambio = float(fila.get_text().split("¢")[1][0:5])
    
    # Lista para guardar los elementos correspondientes
    lista_precios = []

    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in sopa_bs4.find_all(tipo, class_ = clase):
        # Removemos las comas de los precios
        fila_limpia = fila.get_text().replace(",", "")
        
        # Ponemos el precio, en caso de que esté en dólares se multiplica por el tipo de cambio de la página
        if(fila_limpia.count("¢") > 0):
            coste = float(fila_limpia.split("¢ ")[1][0:8])
        elif(fila_limpia.count("$") > 0):
            coste = float(fila_limpia.split("$")[1][0:5])
            coste = coste * tipo_cambio
        else:
            coste = np.nan
        lista_precios.append(coste)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_precios

In [9]:
# Hay otros precios que no se pueden extraer con la función anterior, por lo que se hará una función para estos últimos
def elemento_precio_b(sopa_bs4, tipo, clase):
    '''
    Función que extrae el precio, en colones, de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_precios : list
        Lista con los precios, en colones, de cada carro encontrados dentro de la página web de crautos
    '''
    # Se inicia extrayendo el tipo de cambio
    tipo_cambio = 0
    
    for fila in sopa_bs4.find_all('div', class_ = 'brandtitle2'):
        if(fila.get_text().count("Tipo de cambio: ") > 0):
            tipo_cambio = float(fila.get_text().split("¢")[1][0:5])
    
    # Lista para guardar los elementos correspondientes
    lista_precios = []
    
    # Para la siguiente parte, se encuentran algunos repetidos, por lo que se hará un set para evitar este problema
    lista_sec = set(sopa_bs4.find_all(tipo, class_ = clase))
    
    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for fila in lista_sec:
        # Removemos las comas de los precios
        fila_limpia = fila.get_text().replace(",", "")
        
        if(fila_limpia.count("¢") > 0):
            coste = float(fila_limpia.split("¢")[1][1:8])
        elif(fila_limpia.count("$") > 0):
            coste = float(fila_limpia.split("$")[1][0:5])
            coste = coste * tipo_cambio
        else:
            coste = np.nan
        lista_precios.append(coste)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_precios

#### Recolección de datos de crautos

Con las funciones anteriores, se procede a obtener todos los elementos para el DataFrame.

In [10]:
# Iniciando por la marca del carro
marcas = elementos_simples(sopa, 'span', 'brandtitle')
marcas_2 = elementos_simples(sopa, 'td', 'brandtitle2')

# Juntamos las 2 listas de marcas
marcas = marcas + marcas_2

# Seguimos con los modelos
modelos = elemento_modelos(sopa, 'td', 'modeltitle')

# Continuamos con los años
annos = elemento_anno(sopa, 'td', 'modeltitle')

# Luego con la transmisión
transmisiones = elemento_transmision(sopa, 'div', 'transtitle')

# Extraemos el precio
precio = elemento_precio(sopa, 'td', 'precio')
precio_2 = elemento_precio_b(sopa, 'span', 'precio')

# Juntamos las listas de precios
precio = precio + precio_2

El DataFrame de crautos es el siguiente.

In [11]:
df_carros = pd.DataFrame(
    {"Marca": marcas,
     "Modelo": modelos, 
     "Años": annos, 
     "Transmisiones": transmisiones[0:62],
     "Precio": precio[0:62]})

# Se agrega un indicador para diferenciar posteriormente cada página web
df_carros['Origen'] = 'crautos'

df_carros.tail()

,Marca,Modelo,Años,Transmisiones,Precio,Origen
57,Renault,DUSTER,2013,Automático,2000000.0,crautos
58,Honda,CRV EX,2007,Automático,4150000.0,crautos
59,Hyundai,TUCSON GL,2023,Automático,250000.0,crautos
60,Land Rover,DISCOVERY SE,2017,Manual,8490000.0,crautos
61,Nissan,QASHQAI,2021,Manual,1850000.0,crautos


#### Extracción de Mercado libre

Con uno de los DataFrames ya creados, se procede a hacer uno similar con otra página de [compra/venta de automóviles](https://autos.mercadolibre.co.cr/autos-camionetas/usados/autos-usados), para esto se extraerán los mismos elementos que el el DataFrame anterior para compararlo.

Para esto, se buscó sobre la forma de obtener las entradas [impares](https://j2logo.com/python/tutorial/tipo-list-python/#list-methods) de una lista con ::, es decir, la entrada 1, 3, 5, ...

In [12]:
# Se selecciona el url del que se extraerá la información
url = "https://autos.mercadolibre.co.cr/autos-camionetas/usados/autos-usados"

# Manda una solicitud para obtener la información de la página web (librería requests)
respuesta = requests.get(url)

# Se analiza la información del HTML (librería BeautifulSoup)
sopa = BeautifulSoup(respuesta.content, 'html.parser')

Se hará una función para extraer el precio, para comparar de manera justa, se usará el tipo de cambio de crautos para aquellos automóviles que tengan el precio en dólares, esto ya que MercadoLibre no incorpora uno en su página.

In [13]:
def precio_final(sopa_bs4):
    '''
    Función que extrae el precio, en colones, de cada carro de la página web de crautos

    Parameters
    sopa : bs4.BeautifulSoup
        HTML leído a trevés de la librería BeautifulSoup
    tipo : str
        Tipo de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup
    clase : str
        Clase de elemento a buscar y extraer de un elemento 'sopa' del paquete BeautifulSoup

    Returns
    lista_precios : list
        Lista con los precios, en colones, de cada carro encontrados dentro de la página web de crautos
    '''
    # Se inicia extrayendo el tipo de cambio de la página de crautos
    url = "https://crautos.com/autosusados/"
    
    # Manda una solicitud para obtener la información de la página web (librería requests)
    respuesta = requests.get(url)
    
    # Se analiza la información del HTML (librería BeautifulSoup)
    sopa_tc = BeautifulSoup(respuesta.content, 'html.parser')
    
    tipo_cambio = 0
    
    for fila in sopa_tc.find_all('div', class_ = 'brandtitle2'):
        if(fila.get_text().count("Tipo de cambio: ") > 0):
            tipo_cambio = float(fila.get_text().split("¢")[1][0:5])
    
    # Lista para guardar los elementos correspondientes
    lista_precios = []
    
    # Se extraen el monto y la moneda por separado
    moneda = elementos_simples(sopa_bs4, 'span', 'andes-money-amount__currency-symbol')
    monto = elementos_simples(sopa_bs4, 'span', 'andes-money-amount__fraction')
    
    # Se itera por cada uno de los elementos del HTML con la clase y el tipo especificado
    for iterador in range(len(monto)):
        # Removemos las comas de los precios
        cantidad = monto[iterador].replace(",", "")

        # Separamos los casos
        if(moneda[iterador].count("₡") > 0):
            coste = float(cantidad)
        elif(moneda[iterador].count("US$") > 0):
            coste = float(cantidad)
            coste = coste * tipo_cambio
        else:
            coste = np.nan
        lista_precios.append(coste)

        # Se deja un espacio de un segundo entre cada iteración, esto para no sobrecargar la página, pues en cada iteración se mete a ella
        time.sleep(1)
    return lista_precios

In [14]:
# Se extraen la marca y el modelo
modelos_unidos = elementos_simples(sopa, 'h2', 'poly-box')

# La primera palabra es la marca, el resto es el modelo, por lo que se separa cada una
marcas = [entrada.split()[0] for entrada in modelos_unidos]
modelos = [" ".join(entrada.split()[1:]) for entrada in modelos_unidos]

# Se extrae el año del vehículo
annos = elementos_simples(sopa, 'li', 'poly-attributes-list__item poly-attributes-list__bar')

# Se dejan solo las entradas impares (1, 3, 5, 7, ...) pues las pares contienen el kilometraje, el cuál no se ocupa para este proceso
annos = annos[::2]

# Se obtienen los precios
precios = precio_final(sopa)

Repetiremos el proceso con la segunda página de Mercado libre, para tener una muestra más grande.

In [15]:
# Se selecciona el url del que se extraerá la información
url = "https://autos.mercadolibre.co.cr/autos-camionetas/usados/autos-usados_Desde_49_NoIndex_True"

# Manda una solicitud para obtener la información de la página web (librería requests)
respuesta = requests.get(url)

# Se analiza la información del HTML (librería BeautifulSoup)
sopa = BeautifulSoup(respuesta.content, 'html.parser')

In [16]:
# Se extraen la marca y el modelo
modelos_unidos_2 = elementos_simples(sopa, 'h2', 'poly-box')

# La primera palabra es la marca, el resto es el modelo, por lo que se separa cada una
marcas_2 = [entrada.split()[0] for entrada in modelos_unidos_2]
modelos_2 = [" ".join(entrada.split()[1:]) for entrada in modelos_unidos_2]

# Se extrae el año del vehículo
annos_2 = elementos_simples(sopa, 'li', 'poly-attributes-list__item poly-attributes-list__bar')

# Se dejan solo las entradas impares (1, 3, 5, 7, ...) pues las pares contienen el kilometraje, el cuál no se ocupa para este proceso
annos_2 = annos_2[::2]

# Se obtienen los precios
precios_2 = precio_final(sopa)

In [17]:
# Se juntan las listas anteriores
marcas = marcas + marcas_2
modelos = modelos + modelos_2
annos = annos + annos_2
precios = precios + precios_2

In [18]:
df_carros_b = pd.DataFrame(
    {"Marca": marcas,
     "Modelo": modelos, 
     "Años": annos,
     "Precio": precios}
)

df_carros_b['Origen'] = 'Mercado libre'

df_carros_b.tail()

,Marca,Modelo,Años,Precio,Origen
91,Mitsubishi,Montero Sport 4x4,2017,18250000.0,Mercado libre
92,Toyota,Fortuner Manua,2007,18500.0,Mercado libre
93,Toyota,Yaris Nacional,2011,5900000.0,Mercado libre
94,Isuzu,Dmax 2017 4x2 D-max Dmax Turbo,2017,10800000.0,Mercado libre
95,No,Disponible Contenedor Estruct A Carga,1999,1350000.0,Mercado libre


In [5]:
# Se guardan los DataFrames en excel para añadir más observaciones el día siguiente, estos se comentan para no guardar archivos en cada ejecución
# df_carros.to_excel('autos_crautos.xlsx', index = False)
# df_carros_b.to_excel('autos_mercado_libre.xlsx', index = False)

### Análisis con los datos

Para iniciar con el análisis respectivo, se unirán los 2 DataFrames creados.